In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import requests
import gzip
import os
import json # JSONを扱うためにインポート

# --- 1. データセットの準備と有向グラフの読み込み (変更なし) ---
def download_and_load_digraph():
    """SNAPのego-Twitterデータセットをダウンロードし、有向グラフとして読み込む"""
    url = "https://snap.stanford.edu/data/twitter_combined.txt.gz"
    gz_filename = "twitter_combined.txt.gz"
    txt_filename = "twitter_combined.txt"

    if not os.path.exists(txt_filename):
        print(f"データセット '{txt_filename}' をダウンロードします...")
        try:
            r = requests.get(url, stream=True)
            r.raise_for_status()
            with open(gz_filename, "wb") as f:
                f.write(r.content)
            with gzip.open(gz_filename, 'rb') as f_in:
                with open(txt_filename, 'wb') as f_out:
                    f_out.write(f_in.read())
            os.remove(gz_filename)
            print("ダウンロードと解凍が完了しました。")
        except requests.exceptions.RequestException as e:
            print(f"エラー: ダウンロードに失敗しました。 {e}")
            return None
            
    print("有向グラフをファイルから読み込んでいます...")
    G = nx.read_edgelist(txt_filename, create_using=nx.DiGraph())
    return G

# --- 2. PageRankとBFSによるサンプリング (変更なし) ---
def pagerank_bfs_sampling_directed(G, target_node_count=20000, nodes_per_seed=200):
    """PageRankでシードを選び、BFSで近傍を探索して有向グラフをサンプリングする"""
    if G is None: return None
    print("PageRankスコアを計算中...")
    pagerank_scores = nx.pagerank(G, alpha=0.85)
    sorted_nodes_by_pagerank = sorted(pagerank_scores.keys(), key=lambda n: pagerank_scores[n], reverse=True)

    print("PageRank上位ノードをシードとしてBFSサンプリングを開始します...")
    sampled_nodes = set()
    
    for seed_node in sorted_nodes_by_pagerank:
        if seed_node in sampled_nodes: continue
        
        bfs_nodes = {seed_node}
        queue = list(G.successors(seed_node)) 
        
        head = 0
        while head < len(queue) and len(bfs_nodes) < nodes_per_seed:
            current_node = queue[head]
            head += 1
            if current_node not in bfs_nodes:
                bfs_nodes.add(current_node)
                for successor in G.successors(current_node):
                    if successor not in bfs_nodes:
                        queue.append(successor)

        sampled_nodes.update(bfs_nodes)

        if len(sampled_nodes) >= target_node_count: break
            
    print("誘導サブグラフを生成中...")
    G_sub = G.subgraph(list(sampled_nodes)).copy()
    
    isolated = [n for n, d in G_sub.degree() if d == 0]
    G_sub.remove_nodes_from(isolated)
    
    return G_sub

# --- 3. グラフの保存 (機能追加) ---
def save_graph_to_files(G, txt_filename="sampled_graph.txt", json_filename="sampled_graph.json"):
    """グラフをTXT(エッジリスト)形式とJSON(隣接リスト)形式で保存する"""
    if G is None:
        print("グラフが空のため、保存をスキップします。")
        return

    # TXT形式 (エッジリスト) で保存
    print(f"\nグラフをTXT形式で '{txt_filename}' に保存しています...")
    try:
        # `delimiter`で区切り文字を指定
        nx.write_edgelist(G, txt_filename, data=False, delimiter=' ')
        print(f"'{txt_filename}' の保存が完了しました。")
    except Exception as e:
        print(f"エラー: TXTファイルの保存に失敗しました。 {e}")

    # JSON形式 (隣接リスト) で保存
    print(f"グラフをJSON形式で '{json_filename}' に保存しています...")
    try:
        # NetworkXの機能で隣接リスト形式の辞書を生成
        adj_data = nx.to_dict_of_lists(G)
        with open(json_filename, 'w') as f:
            # `indent=4`できれいに整形して書き込む
            json.dump(adj_data, f, indent=4)
        print(f"'{json_filename}' の保存が完了しました。")
    except Exception as e:
        print(f"エラー: JSONファイルの保存に失敗しました。 {e}")


# --- 4. グラフ特性の比較 (可視化部分はコメントアウト) ---
def analyze_graph_properties(G_original, G_sub):
    """グラフの基本情報を表示し、次数分布を比較する（プロットは省略）"""
    print("\n--- サンプリング結果の分析 ---")
    print(f"元のグラフ: ノード数 {G_original.number_of_nodes()}, エッジ数 {G_original.number_of_edges()}")
    print(f"サブグラフ: ノード数 {G_sub.number_of_nodes()}, エッジ数 {G_sub.number_of_edges()}")
    # ここに必要であれば、以前のコードにあったmatplotlibでのプロット関数呼び出しを追加できます。


# --- メイン処理 (変更あり) ---
if __name__ == "__main__":
    G_original = download_and_load_digraph()
    
    if G_original:
        # サンプリングを実行
        G_sub = pagerank_bfs_sampling_directed(G_original, target_node_count=20000)
        
        if G_sub and G_sub.number_of_nodes() > 0:
            # グラフの基本情報を表示
            analyze_graph_properties(G_original, G_sub)
            
            # グラフを指定した形式でファイルに保存
            save_graph_to_files(G_sub, txt_filename="sampled_twitter_graph.txt", json_filename="sampled_twitter_graph.json")
        else:
            print("サンプリングの結果、有効なグラフが生成されませんでした。")

データセット 'twitter_combined.txt' をダウンロードします...
ダウンロードと解凍が完了しました。
有向グラフをファイルから読み込んでいます...
PageRankスコアを計算中...
PageRank上位ノードをシードとしてBFSサンプリングを開始します...
誘導サブグラフを生成中...

--- サンプリング結果の分析 ---
元のグラフ: ノード数 81306, エッジ数 1768149
サブグラフ: ノード数 20021, エッジ数 626073

グラフをTXT形式で 'sampled_twitter_graph.txt' に保存しています...
'sampled_twitter_graph.txt' の保存が完了しました。
グラフをJSON形式で 'sampled_twitter_graph.json' に保存しています...
'sampled_twitter_graph.json' の保存が完了しました。


In [2]:
import networkx as nx
import os

# --- グラフをファイルから読み込む ---
filename = "sampled_twitter_graph.txt"

if not os.path.exists(filename):
    print(f"エラー: ファイル '{filename}' が見つかりません。")
    print("先にサンプリングを実行してグラフファイルを生成してください。")
else:
    # create_using=nx.DiGraph を指定して、有向グラフとして正しく読み込む
    G_sampled = nx.read_edgelist(filename, create_using=nx.DiGraph())
    print(f"'{filename}' を有向グラフとして読み込みました。")
    print(f"  ノード数: {G_sampled.number_of_nodes()}")
    print(f"  エッジ数: {G_sampled.number_of_edges()}")
    
    print("\n--- 有向グラフであるかの検証 ---")

    # --- 検証1: オブジェクトの型を確認 ---
    is_digraph_instance = isinstance(G_sampled, nx.DiGraph)
    print(f"1. オブジェクトは nx.DiGraph のインスタンスか？ -> {is_digraph_instance}")
    if is_digraph_instance:
        print("   => 成功: グラフは有向グラフクラスのオブジェクトです。")
    else:
        print("   => 失敗: グラフは有向グラフクラスのオブジェクトではありません。")

    # --- 検証2: is_directed() メソッドを使用 ---
    is_directed_by_method = G_sampled.is_directed()
    print(f"\n2. G.is_directed() メソッドの結果は？ -> {is_directed_by_method}")
    if is_directed_by_method:
        print("   => 成功: メソッドがグラフを有向であると判定しました。")
    else:
        print("   => 失敗: メソッドがグラフを有向でないと判定しました。")

    # --- 検証3: 非対称なエッジ（片思いのフォロー関係）を探す ---
    print("\n3. 非対称なエッジ（片方向のリンク）を探します...")
    found_asymmetric_edge = False
    # グラフ内のすべてのエッジをループして調べる
    for u, v in G_sampled.edges():
        # (u, v) のエッジに対し、逆方向の (v, u) が存在しない場合
        if not G_sampled.has_edge(v, u):
            print(f"   => 発見: ノード {u} は {v} をフォローしていますが、")
            print(f"           ノード {v} は {u} をフォローしていません。")
            found_asymmetric_edge = True
            break # 1つ見つかれば十分なのでループを抜ける
    
    if not found_asymmetric_edge:
        print("   => 非対称なエッジは見つかりませんでした（グラフが小さい場合や特殊な場合）。")

    print("\n--- 結論 ---")
    if is_digraph_instance and is_directed_by_method and found_asymmetric_edge:
        print("✅ すべての検証により、サンプリングされたグラフは正しく有向グラフとして扱われています。")
    else:
        print("❌ 検証に失敗した項目があります。コードやデータを確認してください。")

'sampled_twitter_graph.txt' を有向グラフとして読み込みました。
  ノード数: 20021
  エッジ数: 626073

--- 有向グラフであるかの検証 ---
1. オブジェクトは nx.DiGraph のインスタンスか？ -> True
   => 成功: グラフは有向グラフクラスのオブジェクトです。

2. G.is_directed() メソッドの結果は？ -> True
   => 成功: メソッドがグラフを有向であると判定しました。

3. 非対称なエッジ（片方向のリンク）を探します...
   => 発見: ノード 16513135 は 13104722 をフォローしていますが、
           ノード 13104722 は 16513135 をフォローしていません。

--- 結論 ---
✅ すべての検証により、サンプリングされたグラフは正しく有向グラフとして扱われています。
